### 1.融合特征 模型AUC下降--异或关系

In [25]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score

# 模拟数据
np.random.seed(42)
tf.random.set_seed(42)

num_samples = 10000

# 基础特征
age = np.random.randint(18, 60, num_samples)
gender = np.random.randint(0, 2, num_samples)
device_type = np.random.randint(0, 3, num_samples)

# 广告特征与复购特征
ad_click_rate = np.random.uniform(0.1, 0.9, num_samples)
repurchase_freq = np.random.uniform(0.1, 0.9, num_samples)

# 构造标签：广告高 XOR 复购高（正样本只能满足其中一个条件）
label = ((ad_click_rate > 0.6) ^ (repurchase_freq > 0.6)).astype(np.int32)

# 引入更强的噪声：30% 的正样本被随机翻转
positive_indices = np.where(label == 1)[0]
num_noise = int(len(positive_indices) * 0.3)  # 将噪声比例从 30% 提高到 30%
noise_idx = np.random.choice(positive_indices, size=num_noise, replace=False)
label[noise_idx] = 0  # 将部分正样本标记为负样本

# 添加更多干扰特征（与目标无关）
random_noise1 = np.random.normal(0, 1, num_samples)
random_noise2 = np.random.normal(0, 1, num_samples)

# 分割数据
x_base = np.stack([age, gender, device_type, random_noise1, random_noise2], axis=1)
x_ad = np.stack([ad_click_rate, random_noise1, random_noise2], axis=1)
x_repurchase = np.stack([repurchase_freq, random_noise1, random_noise2], axis=1)
x_combined = np.stack([ad_click_rate, repurchase_freq, random_noise1, random_noise2], axis=1)



# 模型定义（极简线性模型）
def build_model(input_shape):
    inputs = tf.keras.Input(shape=(input_shape,))
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(inputs)  # 使用线性模型
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc')])
    return model

# 训练函数
def train_model(x, y, feature_name):
    model = build_model(x.shape[1])
    model.fit(x, y, epochs=5, batch_size=64, validation_split=0.2, verbose=0)
    y_pred = model.predict(x).flatten()
    auc = roc_auc_score(y, y_pred)
    print(f"{feature_name} AUC: {auc:.4f}")
    return auc

# 基线模型（仅基础特征）
base_auc = train_model(x_base, label, "Base Model")

# 单独加入广告特征
ad_auc = train_model(x_ad, label, "Ad Feature Only")

# 单独加入复购特征
repurchase_auc = train_model(x_repurchase, label, "Repurchase Feature Only")

# 同时加入广告+复购特征
combined_auc = train_model(x_combined, label, "Ad + Repurchase Features")


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step
Base Model AUC: 0.4952
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step
Ad Feature Only AUC: 0.5596
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step
Repurchase Feature Only AUC: 0.5251
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step
Ad + Repurchase Features AUC: 0.4754


### 2.头部反向关系

In [23]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score

# 模拟数据
np.random.seed(42)
tf.random.set_seed(42)

num_samples = 10000

# 基础特征
age = np.random.randint(18, 60, num_samples)
gender = np.random.randint(0, 2, num_samples)
device_type = np.random.randint(0, 3, num_samples)

# 广告特征与复购特征
ad_click_rate = np.random.uniform(0.1, 0.9, num_samples)
repurchase_freq = np.random.uniform(0.1, 0.9, num_samples)

# 构造标签：强制广告高或复购高 → 正样本，其他 → 负样本
label = np.zeros(num_samples)
label[ad_click_rate > 0.7] = 1
label[repurchase_freq > 0.7] = 1
# 强制广告+复购高 → 负样本
label[(ad_click_rate > 0.7) & (repurchase_freq > 0.7)] = 0
# 强制广告+复购低 → 负样本（可选）
label[(ad_click_rate < 0.3) & (repurchase_freq < 0.3)] = 0

# 引入更强的噪声：10% 的正样本被翻转
positive_indices = np.where(label == 1)[0]
num_noise = int(len(positive_indices) * 0.1)
noise_idx = np.random.choice(positive_indices, size=num_noise, replace=False)
label[noise_idx] = 0

# 添加干扰特征
random_noise = np.random.normal(0, 1, (num_samples, 5))

# 构造输入数据
x_base = np.stack([age, gender, device_type] + [random_noise[:, i] for i in range(5)], axis=1)
x_ad = np.stack([ad_click_rate] + [random_noise[:, i] for i in range(5)], axis=1)
x_repurchase = np.stack([repurchase_freq] + [random_noise[:, i] for i in range(5)], axis=1)
x_combined = np.stack([ad_click_rate, repurchase_freq] + [random_noise[:, i] for i in range(5)], axis=1)

# 构造极端不平衡数据（正样本占比 10%）
positive_indices = np.where(label == 1)[0]
negative_indices = np.where(label == 0)[0]

# 计算需要选择的负样本数量
required_neg_samples = int(len(positive_indices) * 9)
if len(negative_indices) < required_neg_samples:
    # 如果负样本不足，允许重复采样（replace=True）
    selected_negative = np.random.choice(negative_indices, size=required_neg_samples, replace=True)
else:
    selected_negative = np.random.choice(negative_indices, size=required_neg_samples, replace=False)

final_indices = np.concatenate([positive_indices, selected_negative])
x_final = x_combined[final_indices]
y_final = label[final_indices]

# 模型定义（极简线性模型）
def build_model(input_shape):
    inputs = tf.keras.Input(shape=(input_shape,))
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc')])
    return model

# 训练函数
def train_model(x, y, feature_name):
    model = build_model(x.shape[1])
    model.fit(x, y, epochs=5, batch_size=64, validation_split=0.2, verbose=0)
    y_pred = model.predict(x).flatten()
    auc = roc_auc_score(y, y_pred)
    print(f"{feature_name} AUC: {auc:.4f}")
    return auc

# 基线模型（仅基础特征）
base_auc = train_model(x_base[final_indices], y_final, "Base Model")

# 单独加入广告特征
ad_auc = train_model(x_ad[final_indices], y_final, "Ad Feature Only")

# 单独加入复购特征
repurchase_auc = train_model(x_repurchase[final_indices], y_final, "Repurchase Feature Only")

# 同时加入广告+复购特征
combined_auc = train_model(x_final, y_final, "Ad + Repurchase Features")


1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 537us/step
Base Model AUC: 0.4897
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step
Ad Feature Only AUC: 0.3236
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 433us/step
Repurchase Feature Only AUC: 0.3126
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 562us/step
Ad + Repurchase Features AUC: 0.2156


### 3.0 如何优化

In [27]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score

# 模拟数据
np.random.seed(42)
tf.random.set_seed(42)

num_samples = 10000

# 基础特征
age = np.random.randint(18, 60, num_samples)
gender = np.random.randint(0, 2, num_samples)
device_type = np.random.randint(0, 3, num_samples)

# 广告特征与复购特征
ad_click_rate = np.random.uniform(0.1, 0.9, num_samples)
repurchase_freq = np.random.uniform(0.1, 0.9, num_samples)

# 构造标签：广告高 XOR 复购高
label = ((ad_click_rate > 0.6) ^ (repurchase_freq > 0.6)).astype(np.int32)

# 引入噪声：30% 的正样本被翻转
positive_indices = np.where(label == 1)[0]
num_noise = int(len(positive_indices) * 0.4)
noise_idx = np.random.choice(positive_indices, size=num_noise, replace=False)
label[noise_idx] = 0

# 添加干扰特征
random_noise1 = np.random.normal(0, 1, num_samples)
random_noise2 = np.random.normal(0, 1, num_samples)
random_noise3 = np.random.normal(0, 1, num_samples)
random_noise4 = np.random.normal(0, 1, num_samples)
random_noise5 = np.random.normal(0, 1, num_samples)

# 分割数据
x_base = np.stack([age, gender, device_type, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)
x_ad = np.stack([ad_click_rate, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)
x_repurchase = np.stack([repurchase_freq, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)
x_combined = np.stack([ad_click_rate, repurchase_freq, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)



#### 3.1 优化版本1

In [28]:

# 模型定义（基础线性模型）
def build_base_model(input_shape):
    inputs = tf.keras.Input(shape=(input_shape,))
    x = tf.keras.layers.Dense(16, activation='relu')(inputs)
    x = tf.keras.layers.Dense(8, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
#     outputs = tf.keras.layers.Dense(1, activation='sigmoid')(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc')])
    return model

# 修改模型结构：添加广告与复购的交叉特征（仅用于 x_combined）
def build_cross_model(input_shape):
    inputs = tf.keras.Input(shape=(input_shape,))
    
    # 特征交叉（广告 × 复购）
    ad_input = inputs[:, 0:1]  # 广告特征是第 0 维
    repurchase_input = inputs[:, 1:2]  # 复购特征是第 1 维
    cross_feature = ad_input * repurchase_input  # 直接相乘，无需 reshape
    
    # 拼接原始特征与交叉特征
    combined = tf.keras.layers.Concatenate()([inputs, cross_feature])
    
    # DNN 部分
    x = tf.keras.layers.Dense(16, activation='relu')(combined)
    x = tf.keras.layers.Dense(8, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc')])
    return model

# 训练函数
def train_model(x, y, feature_name, model_builder):
    model = model_builder(x.shape[1])
    model.fit(x, y, epochs=10, batch_size=64, validation_split=0.2, verbose=0)
    y_pred = model.predict(x).flatten()
    auc = roc_auc_score(y, y_pred)
    print(f"{feature_name} AUC: {auc:.4f}")
    return auc

# 基线模型（线性模型）
base_auc = train_model(x_base, label, "Base Model", build_base_model)

# 单独加入广告特征
ad_auc = train_model(x_ad, label, "Ad Feature Only", build_base_model)


# 单独加入复购特征
repurchase_auc = train_model(x_repurchase, label, "Repurchase Feature Only", build_base_model)

# 同时加入广告+复购特征
combined_auc_v1 = train_model(x_combined, label, "Ad + Repurchase Features V1", build_base_model)

# 同时加入广告+复购特征（使用优化后的模型）
combined_auc_v2 = train_model(x_combined, label, "Ad + Repurchase Features V2", build_cross_model)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step
Base Model AUC: 0.5074
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step
Ad Feature Only AUC: 0.5634
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step
Repurchase Feature Only AUC: 0.5690
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step
Ad + Repurchase Features V1 AUC: 0.5986
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step
Ad + Repurchase Features V2 AUC: 0.6568


In [29]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score

# # 模拟数据
# np.random.seed(42)
# tf.random.set_seed(42)

# num_samples = 10000

# # 基础特征
# age = np.random.randint(18, 60, num_samples)
# gender = np.random.randint(0, 2, num_samples)
# device_type = np.random.randint(0, 3, num_samples)

# # 广告特征与复购特征
# ad_click_rate = np.random.uniform(0.1, 0.9, num_samples)
# repurchase_freq = np.random.uniform(0.1, 0.9, num_samples)

# # 构造标签：广告高 XOR 复购高
# label = ((ad_click_rate > 0.6) ^ (repurchase_freq > 0.6)).astype(np.int32)

# # 引入噪声：30% 的正样本被翻转
# positive_indices = np.where(label == 1)[0]
# num_noise = int(len(positive_indices) * 0.4)
# noise_idx = np.random.choice(positive_indices, size=num_noise, replace=False)
# label[noise_idx] = 0

# # 添加干扰特征
# random_noise1 = np.random.normal(0, 1, num_samples)
# random_noise2 = np.random.normal(0, 1, num_samples)
# random_noise3 = np.random.normal(0, 1, num_samples)
# random_noise4 = np.random.normal(0, 1, num_samples)
# random_noise5 = np.random.normal(0, 1, num_samples)

# # 分割数据
# x_base = np.stack([age, gender, device_type, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)
# x_ad = np.stack([ad_click_rate, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)
# x_repurchase = np.stack([repurchase_freq, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)
# x_combined = np.stack([ad_click_rate, repurchase_freq, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)



# 模型定义（优化后）
def build_base_model(input_shape, initial_learning_rate=0.001, decay_steps=500):
    inputs = tf.keras.Input(shape=(input_shape,))
    x = Dense(16, activation='relu')(inputs)
    x = Dropout(0.2)(x)  # 添加正则化
    x = Dense(8, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=initial_learning_rate,  # 修正参数名
        decay_steps=decay_steps,
        alpha=0.1  # 最终学习率是初始的10%
    )
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )
    return model

def build_cross_model(input_shape, initial_learning_rate=0.001, decay_steps=500):
    inputs = Input(shape=(input_shape,))
    ad_input = inputs[:, 0:1]
    repurchase_input = inputs[:, 1:2]
    cross_feature = ad_input * repurchase_input
    combined = concatenate([inputs, cross_feature])
    
    x = Dense(16, activation='relu')(combined)
    x = Dropout(0.2)(x)  # 添加正则化
    x = Dense(8, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=initial_learning_rate,  # 修正参数名
        decay_steps=decay_steps,
        alpha=0.1
    )
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )
    return model

# 训练函数（带早停）
def train_model(x, y, feature_name, model_builder, initial_lr=0.001):
    epochs = 50
    batch_size = 64
    steps_per_epoch = len(x) // batch_size
    decay_steps = epochs * steps_per_epoch
    
    model = model_builder(
        input_shape=x.shape[1],
        initial_learning_rate=initial_lr,  # 确保参数名一致
        decay_steps=decay_steps
    )
    
    callbacks = [
        EarlyStopping(
            monitor='val_auc',
            patience=10,
            restore_best_weights=True,
            mode='max'
        )
    ]
    
    model.fit(
        x, y,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=callbacks,
        verbose=0
    )
    
    y_pred = model.predict(x).flatten()
    auc = roc_auc_score(y, y_pred)
    print(f"{feature_name} AUC: {auc:.4f}")
    return auc

# 训练模型（使用余弦退火）
base_auc = train_model(x_base, label, "Base Model", build_base_model)
ad_auc = train_model(x_ad, label, "Ad Feature Only", build_base_model)
repurchase_auc = train_model(x_repurchase, label, "Repurchase Feature Only", build_base_model)
combined_auc_v1 = train_model(x_combined, label, "Ad + Repurchase Features V1", build_base_model)
combined_auc_v2 = train_model(x_combined, label, "Ad + Repurchase Features V2", build_cross_model)


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step
Base Model AUC: 0.5120
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step
Ad Feature Only AUC: 0.5912
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step
Repurchase Feature Only AUC: 0.6037
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step
Ad + Repurchase Features V1 AUC: 0.8608
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step
Ad + Repurchase Features V2 AUC: 0.8602


#### 3.2 优化版本2-打印学习率

In [30]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score

# # 模拟数据（保持不变）
# np.random.seed(42)
# tf.random.set_seed(42)

# num_samples = 10000

# age = np.random.randint(18, 60, num_samples)
# gender = np.random.randint(0, 2, num_samples)
# device_type = np.random.randint(0, 3, num_samples)
# ad_click_rate = np.random.uniform(0.1, 0.9, num_samples)
# repurchase_freq = np.random.uniform(0.1, 0.9, num_samples)
# label = ((ad_click_rate > 0.6) ^ (repurchase_freq > 0.6)).astype(np.int32)

# positive_indices = np.where(label == 1)[0]
# num_noise = int(len(positive_indices) * 0.4)
# noise_idx = np.random.choice(positive_indices, size=num_noise, replace=False)
# label[noise_idx] = 0

# random_noise1 = np.random.normal(0, 1, num_samples)
# random_noise2 = np.random.normal(0, 1, num_samples)
# random_noise3 = np.random.normal(0, 1, num_samples)
# random_noise4 = np.random.normal(0, 1, num_samples)
# random_noise5 = np.random.normal(0, 1, num_samples)

# x_base = np.stack([age, gender, device_type, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)
# x_ad = np.stack([ad_click_rate, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)
# x_repurchase = np.stack([repurchase_freq, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)
# x_combined = np.stack([ad_click_rate, repurchase_freq, random_noise1, random_noise2, random_noise3, random_noise4, random_noise5], axis=1)

# 自定义 Callback：打印每个 epoch 的学习率
class LearningRatePrinter(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # 修正：使用 learning_rate 而非 lr
        current_lr = self.model.optimizer.learning_rate.numpy()  # 关键修改
        if epoch % 5 == 0:
            print(f"Epoch {epoch+1}: Learning Rate = {current_lr:.6f}")

# 模型定义（保持不变，仅添加注释）
def build_base_model(input_shape, initial_learning_rate=0.001, decay_steps=500):
    inputs = tf.keras.Input(shape=(input_shape,))
    x = Dense(16, activation='relu')(inputs)
    x = Dropout(0.2)(x)
    x = Dense(8, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=decay_steps,
        alpha=0.1
    )
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )
    return model

def build_cross_model(input_shape, initial_learning_rate=0.001, decay_steps=500):
    inputs = Input(shape=(input_shape,))
    ad_input = inputs[:, 0:1]
    repurchase_input = inputs[:, 1:2]
    cross_feature = ad_input * repurchase_input
    combined = concatenate([inputs, cross_feature])
    
    x = Dense(16, activation='relu')(combined)
    x = Dropout(0.2)(x)
    x = Dense(8, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=decay_steps,
        alpha=0.1
    )
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )
    return model

# 修改训练函数：添加学习率打印回调
def train_model(x, y, feature_name, model_builder, initial_lr=0.001):
    epochs = 50
    batch_size = 64
    steps_per_epoch = len(x) // batch_size
    decay_steps = epochs * steps_per_epoch
    
    model = model_builder(
        input_shape=x.shape[1],
        initial_learning_rate=initial_lr,
        decay_steps=decay_steps
    )
    
    callbacks = [
        EarlyStopping(
            monitor='val_auc',
            patience=10,
            restore_best_weights=True,
            mode='max'
        ),
        LearningRatePrinter()  # 添加自定义回调
    ]
    
    model.fit(
        x, y,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=callbacks,
        verbose=0
    )
    
    y_pred = model.predict(x).flatten()
    auc = roc_auc_score(y, y_pred)
    print(f"{feature_name} AUC: {auc:.4f}")
    return auc

# 训练模型（使用余弦退火）
base_auc = train_model(x_base, label, "Base Model", build_base_model)
ad_auc = train_model(x_ad, label, "Ad Feature Only", build_base_model)
repurchase_auc = train_model(x_repurchase, label, "Repurchase Feature Only", build_base_model)
combined_auc_v1 = train_model(x_combined, label, "Ad + Repurchase Features V1", build_base_model)
combined_auc_v2 = train_model(x_combined, label, "Ad + Repurchase Features V2", build_cross_model)


Epoch 1: Learning Rate = 0.000999
Epoch 6: Learning Rate = 0.000980
Epoch 11: Learning Rate = 0.000933
Epoch 16: Learning Rate = 0.000862
Epoch 21: Learning Rate = 0.000771
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step
Base Model AUC: 0.5067
Epoch 1: Learning Rate = 0.000999
Epoch 6: Learning Rate = 0.000980
Epoch 11: Learning Rate = 0.000933
Epoch 16: Learning Rate = 0.000862
Epoch 21: Learning Rate = 0.000771
Epoch 26: Learning Rate = 0.000666
Epoch 31: Learning Rate = 0.000555
Epoch 36: Learning Rate = 0.000442
Epoch 41: Learning Rate = 0.000337
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step
Ad Feature Only AUC: 0.5884
Epoch 1: Learning Rate = 0.000999
Epoch 6: Learning Rate = 0.000980
Epoch 11: Learning Rate = 0.000933
Epoch 16: Learning Rate = 0.000862
Epoch 21: Learning Rate = 0.000771
Epoch 26: Learning Rate = 0.000666
Epoch 31: Learning Rate = 0.000555
Epoch 36: Learning Rate = 0.000442
Epoch 41: Learning Rate = 0.000337
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step
Repurchase Feature On

In [41]:
import tensorflow as tf

# 定义参数
learning_rate = 1e-4
decay_steps = 1000

# # 使用TensorFlow 2.x的Variable
# global_step = tf.Variable(0, trainable=False, name='global_step')

# # 创建余弦退火学习率张量（使用tf.keras版本）
# cosine_decay = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate=learning_rate,
#     decay_steps=decay_steps,
#     alpha=0.0
# )

# 设置global_step的值并计算学习率
global_step.assign(500)
result = cosine_decay(global_step).numpy()

print("learning_rate:", learning_rate)
print("lr_decayed:", result)


learning_rate: 0.0001
lr_decayed: 4.9999995e-05
